In [23]:
import json
import urllib3
import time
import urllib.request

In [34]:
def queryBuilder(q1):
    url   = 'http://moe.eecs.qmul.ac.uk:8888/api/v1/query?'
    query = 'query='+q1
    return url+query

#query_range?query=up&start=2015-07-01T20:10:30.781Z&end=2015-07-01T20:11:00.781Z&step=15s
def queryRangeBuilder(query,start,end):
    url   = 'http://moe.eecs.qmul.ac.uk:8888/api/v1/query_range?'
    query = 'query='+ query
    start = 'start='+ start + ".000Z"
    end   = 'end='  + end   + ".000Z"
    steps = 'step=10s' 
    return url+query+"&"+start+"&"+end+"&"+steps


In [35]:



response = wp = urllib.request.urlopen(queryRangeBuilder('akka_system_dead_letters_total',"2019-05-31T13:49:10","2019-05-31T13:58:58"))
parsed = json.load(response)["data"]["result"]

print(parsed)

[{'metric': {'__name__': 'akka_system_dead_letters_total', 'instance': '10.0.0.211:11600', 'job': 'prometheus', 'system': 'dockerexp'}, 'values': [[1559310550, '0'], [1559310560, '0'], [1559310570, '0'], [1559310580, '0'], [1559310590, '0'], [1559310600, '0'], [1559310610, '0'], [1559310620, '0'], [1559310630, '0'], [1559310640, '0'], [1559310650, '0'], [1559310660, '0'], [1559310670, '0'], [1559310680, '0'], [1559310690, '0'], [1559310700, '0'], [1559310710, '0'], [1559310720, '0'], [1559310730, '0'], [1559310740, '0'], [1559310750, '0'], [1559310760, '0'], [1559310770, '0'], [1559310780, '0'], [1559310790, '0'], [1559310800, '0'], [1559310810, '0'], [1559310820, '0'], [1559310830, '0'], [1559310840, '0'], [1559310850, '0'], [1559310860, '0'], [1559310870, '0'], [1559310880, '0'], [1559310890, '0'], [1559310900, '0'], [1559310910, '0'], [1559310920, '0'], [1559310930, '0'], [1559310940, '0'], [1559310950, '0'], [1559310960, '0'], [1559310970, '0'], [1559310980, '0'], [1559310990, '0']